# Import Libraries and Mount Drive

In [1]:
# Import necessary libraries and mount Google Drive
from google.colab import drive
import warnings
import sys
import pandas as pd
import os

# Mount Google Drive to access datasets
drive.mount('/content/drive')

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Add project directory to system path for importing custom modules
sys.path.append('[your path]')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Metadata for Time Series

In [3]:
# Load metadata from JSON files in the 'owid' directory
import json

# Define directory path for datasets
dir_path = '[your path]'

# Initialize dictionary for storing table names
table_names = dict()

# Traverse the directory and load metadata from JSON files
for root, dirs, files in os.walk(os.path.join(dir_path, 'owid')):
    for file in files:
        if file.endswith('.meta.json'):
            with open(os.path.join(root, file)) as f:
                obj = json.load(f)

            # Extract and store the title of the dataset
            title = obj.get('title', obj.get('dataset').get('title'))
            table_names[title] = os.path.join(root, file)

# Define Utility Functions

In [4]:
# Import necessary libraries for numerical operations
import numpy as np
from timeRec.data import *

# Define a function to filter columns in the dataset
def filter_columns(df, columns_to_keep, missing_threshold=0.3, correlation_threshold=0.9):
    """
    Filters columns in the dataset by removing non-numeric, constant, or highly correlated columns.

    Args:
        df (pd.DataFrame): The input dataframe.
        columns_to_keep (list): List of columns to retain, e.g., 'country', 'year'.
        missing_threshold (float): Threshold for missing values (default is 0.3).
        correlation_threshold (float): Threshold for correlation (default is 0.9).

    Returns:
        pd.DataFrame: Filtered dataframe.
    """
    # Drop unnecessary columns
    df_filtered = df.drop(columns=columns_to_keep, errors='ignore')

    # Select only numeric columns
    df_filtered = df_filtered.select_dtypes(include=[np.number])

    # Drop constant columns
    constant_columns = df_filtered.nunique()[df_filtered.nunique() == 1].index
    df_filtered = df_filtered.drop(columns=constant_columns)

    # Drop columns with too many missing values
    df_filtered = df_filtered.dropna(thresh=len(df_filtered) * (1 - missing_threshold), axis=1)

    # Drop highly correlated columns
    correlation_matrix = df_filtered.corr().abs()
    upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    high_corr_features = [col for col in upper_triangle.columns if any(upper_triangle[col] > correlation_threshold)]
    df_filtered = df_filtered.drop(columns=high_corr_features)

    # Add back necessary columns (e.g., 'country')
    df_filtered = pd.concat([df[columns_to_keep], df_filtered], axis=1)

    return df_filtered

# Load and Filter Time Series Data

In [5]:
# Define a function to load and filter time series data
def load_time_series_data(table_names, countries):
    """
    Loads and filters time series data based on the table names dictionary.

    Args:
        table_names (dict): Dictionary mapping dataset titles to paths.
        countries (list): List of countries to filter by.

    Returns:
        dict: Processed time series datasets.
    """
    global total_len_columns, total_year
    time_series_datasets = {}

    for title, path in table_names.items():
        df = pd.read_csv(path.replace('meta.json', 'csv'))

        # Standardize column names for country and location
        if 'country' in df.columns and 'location' in df.columns:
            df = df.drop(columns=['location'])
        elif 'location' in df.columns and 'country' not in df.columns:
            df.rename(columns={'location': 'country'}, inplace=True)

        # Filter by year range and country
        df = df[(df['year'] >= 1970) & (df['year'] <= 2024)]
        df = filter_columns(df, ['country', 'year'], missing_threshold=0.2, correlation_threshold=0.7)
        df = df[df['country'].isin(countries)]

        # Update global counters for columns and years
        total_len_columns += len(df.columns) - 2
        total_year.update(df['year'].unique())

        # Store the filtered DataFrame
        time_series_datasets[title] = df

    return time_series_datasets

# Initialize global variables
total_len_columns = 0
total_year = set()

# Load time series data
time_series_datasets = load_time_series_data(table_names, countries)

print(f"Total number of columns: {total_len_columns}")
print(f"Number of unique years: {len(total_year)}")

KeyboardInterrupt: 

# Merge and Process Time Series Data by Country

In [ ]:
# Import Dask for parallel processing
import dask.dataframe as dd
from tqdm.notebook import tqdm

# Define a function to impute missing data
def impute_missing_data(df, method='linear'):
    """
    Imputes missing data using specified method, excluding 'year' and 'country' columns.

    Args:
        df (pd.DataFrame): DataFrame with missing data.
        method (str): Imputation method ('ffill', 'bfill', 'linear').

    Returns:
        pd.DataFrame: DataFrame with imputed values.
    """
    cols_to_impute = df.columns.difference(['year', 'country'])
    if method == 'ffill':
        df[cols_to_impute] = df[cols_to_impute].ffill()
    elif method == 'bfill':
        df[cols_to_impute] = df[cols_to_impute].bfill()
    elif method == 'linear':
        df[cols_to_impute] = df[cols_to_impute].apply(
            lambda col: col.interpolate(method='linear') if col.dtype in ['float64', 'float32', 'int64', 'int32'] else col
        )
    return df

# Define a function to optimize data types for memory efficiency
def optimize_dtypes(df):
    """
    Optimizes data types for memory efficiency.

    Args:
        df (pd.DataFrame): DataFrame to optimize.

    Returns:
        pd.DataFrame: Optimized DataFrame.
    """
    for col in df.columns:
        if col != 'country':
            if df[col].dtype == 'float64':
                df[col] = df[col].astype('float32')
            elif df[col].dtype == 'int64':
                df[col] = df[col].astype('int32')
    return df

# Define year range from 1970 to 2024
year_range = pd.DataFrame({'year': list(range(1970, 2025))})
npartitions = 50

# Process time series data for each country
for country in tqdm(countries):
    time_series = None
    merge_count = 0

    for title, dataset in time_series_datasets.items():
        country_data = dataset[dataset['country'] == country]
        if country_data.empty:
            continue

        optimized_data = optimize_dtypes(country_data)

        if time_series is None:
            time_series = dd.from_pandas(optimized_data, npartitions=npartitions)
        else:
            new_data = dd.from_pandas(optimized_data, npartitions=npartitions)

            overlap_columns = set(time_series.columns).intersection(set(new_data.columns)) - {'country', 'year'}
            if overlap_columns:
                suffix = f'_right_{merge_count}'
                new_data = new_data.rename(columns={col: col + suffix for col in overlap_columns})

            time_series = dd.merge(time_series, new_data, on=['country', 'year'], how='outer')
            time_series = time_series.persist()
            merge_count += 1

    if time_series is not None:
        # Convert to pandas and merge with full year range
        country_df = time_series.compute()
        country_df = pd.merge(year_range, country_df, on='year', how='left')

        # Remove duplicates and fill missing values
        country_df['non_null_count'] = country_df.notnull().sum(axis=1)
        country_df = country_df.sort_values('non_null_count', ascending=False).drop_duplicates(['country', 'year'], keep='first')
        country_df = country_df.drop(columns=['non_null_count']).sort_values(by='year')

        # Impute missing data and save the result
        country_df = impute_missing_data(country_df, method='linear')
        country_df = impute_missing_data(country_df, method='bfill')

        # Save the processed country data
        os.makedirs(os.path.join(dir_path, 'output'), exist_ok=True)
        country_df.to_csv(os.path.join(dir_path, f'output/{country.replace(" ", "_")}.csv'), index=False)